# 1. 卷积
Conv2d<br>
    Args:
*       in_channels (int): Number of channels in the input image
*       out_channels (int): Number of channels produced by the convolution
*       kernel_size (int or tuple): Size of the convolving kernel
*       stride (int or tuple, optional): Stride of the convolution. Default: 1
*       padding (int, tuple or str, optional): Padding added to all four sides of
            the input. Default: 0
*       padding_mode (str, optional): ``'zeros'``, ``'reflect'``,
            ``'replicate'`` or ``'circular'``. Default: ``'zeros'``
*       dilation (int or tuple, optional): Spacing between kernel elements. Default: 1
*       groups (int, optional): Number of blocked connections from input
*           channels to output channels. Default: 1
*       bias (bool, optional): If ``True``, adds a learnable bias to the
           output. Default: ``True``

In [4]:
import torch
import torch.nn.functional as F

input = torch.tensor([[1, 2, 0, 3, 1],
                      [0, 1, 2, 3, 1],
                      [1, 2, 1, 0, 0],
                      [5, 2, 3, 1, 1],
                      [2, 1, 0, 1, 1]])

kernel = torch.tensor([[1, 2, 1],
                       [0, 1, 0],
                       [2, 1, 0]])

print(input.shape)
print(kernel.shape)
input = torch.reshape(input, (1,1,5,5))     # (minibatch, in_channels, iH, iW)
kernel = torch.reshape(kernel, (1,1,3,3))   # (out_channels, in_channels/groups, kH, kW)
print(input.shape)
print(kernel.shape)

output = F.conv2d(input, kernel, stride=1)
print(f"output:{output}")

output2 = F.conv2d(input, kernel, stride=2)  # 步伐为2
print(f"output2:{output2}")

output3 = F.conv2d(input, kernel, stride=1, padding=1)  # 周围只填充一层
print(f"output3:{output3}")

torch.Size([5, 5])
torch.Size([3, 3])
torch.Size([1, 1, 5, 5])
torch.Size([1, 1, 3, 3])
output:tensor([[[[10, 12, 12],
          [18, 16, 16],
          [13,  9,  3]]]])
output2:tensor([[[[10, 12],
          [13,  3]]]])
output3:tensor([[[[ 1,  3,  4, 10,  8],
          [ 5, 10, 12, 12,  6],
          [ 7, 18, 16, 16,  8],
          [11, 13,  9,  3,  4],
          [14, 13,  9,  7,  4]]]])


In [5]:
import torch
import torchvision
from torch import nn
from torch.nn import Conv2d
from torch.utils.data import DataLoader

dataset = torchvision.datasets.CIFAR10("../dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)       
dataloader = DataLoader(dataset, batch_size=64)
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.conv1 = Conv2d(in_channels=3,out_channels=6,kernel_size=3,stride=1,padding=0) # 彩色图像输入为3层，我们想让它的输出为6层，选3 * 3 的卷积                
    
    def forward(self,x):
        x = self.conv1(x)
        return x
    
tudui = Tudui()
print(tudui)

for data in dataloader:
    imgs, targets = data
    output = tudui(imgs)
    print(imgs.shape)   # 输入为3通道32×32的64张图片
    print(output.shape) # 输出为6通道30×30的64张图片

Files already downloaded and verified
Tudui(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
)
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30]

In [6]:
import torch
import torchvision
from torch import nn
from torch.nn import Conv2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("../dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)       
dataloader = DataLoader(dataset, batch_size=64)
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.conv1 = Conv2d(in_channels=3,out_channels=6,kernel_size=3,stride=1,padding=0) # 彩色图像输入为3层，我们想让它的输出为6层，选3 * 3 的卷积                
    
    def forward(self,x):
        x = self.conv1(x)
        return x
    
tudui = Tudui()
writer = SummaryWriter("logs")
step = 0
for data in dataloader:
    imgs, targets = data
    output = tudui(imgs)
    print(imgs.shape)   
    print(output.shape)    # [64, 6, 30, 30]
    writer.add_images("input", imgs, step)
    output = torch.reshape(output,(-1,3,30,30)) # 把原来6个通道拉为3个通道，为了保证所有维度总数不变，其余的分量分到第一个维度中
    writer.add_images("output", output, step)
    step = step + 1


Files already downloaded and verified
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])
torch.Size

# 2. 池化

In [ ]:
import torch
from torch import nn 
from torch.nn import MaxPool2d

input = torch.tensor([[1,2,0,3,1],
                     [0,1,2,3,1],
                     [1,2,1,0,0],
                     [5,2,3,1,1],
                     [2,1,0,1,1]], dtype = torch.float32)
input = torch.reshape(input,(-1,1,5,5)) 
print(input.shape)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.maxpool = MaxPool2d(kernel_size=3, ceil_mode=True)
        
    def forward(self, input):
        output = self.maxpool(input)
        return output
        
tudui = Tudui()
output = tudui(input)
print(output)

torch.Size([1, 1, 5, 5])
tensor([[[[2., 3.],
          [5., 1.]]]])


In [9]:
import torch
import torchvision
from torch import nn 
from torch.nn import MaxPool2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("../dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)       
dataloader = DataLoader(dataset, batch_size=64)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.maxpool = MaxPool2d(kernel_size=3, ceil_mode=True)
        
    def forward(self, input):
        output = self.maxpool(input)
        return output

tudui = Tudui()
writer = SummaryWriter("logs")
step = 0

for data in dataloader:
    imgs, targets = data
    writer.add_images("pool_input", imgs, step)
    output = tudui(imgs)
    writer.add_images("pool_output", output, step)
    step = step + 1

Files already downloaded and verified


# 3.非线性激活

In [11]:
import torch
from torch import nn
from torch.nn import ReLU

input = torch.tensor([[1,-0.5],
                      [-1,3]])
input = torch.reshape(input,(-1,1,2,2))
print(input.shape)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.relu1 = ReLU()
        
    def forward(self, input):
        output = self.relu1(input)
        return output
    
tudui = Tudui()
output = tudui(input)
print(output)
print(output.shape)

torch.Size([1, 1, 2, 2])
tensor([[[[1., 0.],
          [0., 3.]]]])
torch.Size([1, 1, 2, 2])


In [12]:
import torch
import torchvision
from torch import nn 
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("../dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)       
dataloader = DataLoader(dataset, batch_size=64)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.relu1 = ReLU()
        self.sigmoid1 = Sigmoid()
        
    def forward(self, input):
        output = self.sigmoid1(input)
        return output

tudui = Tudui()
writer = SummaryWriter("logs")
step = 0

for data in dataloader:
    imgs, targets = data
    writer.add_images("sigmoid_input", imgs, step)
    output = tudui(imgs)
    writer.add_images("sigmoid_output", output, step)
    step = step + 1

Files already downloaded and verified


# 4. 线性层

In [ ]:
import torch
import torchvision
from torch import nn 
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("../dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)       
dataloader = DataLoader(dataset, batch_size=64,drop_last=True)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.linear1 = Linear(196608,10)
        
    def forward(self, input):
        output = self.linear1(input)
        return output

tudui = Tudui()
writer = SummaryWriter("logs")
step = 0

for data in dataloader:
    imgs, targets = data
    print(imgs.shape)
    writer.add_images("flatten_input", imgs, step)
    # output = torch.reshape(imgs,(1,1,1,-1)) # 方法一：拉平 ([1, 1, 1, 196608])
    output = torch.flatten(imgs)  # 方法二：拉平。展开为一维 ([196608])
    print(f"reshape output.shape:{output.shape}")   
    output = tudui(output)
    print(f"liner output.shape: {output.shape}")
    step = step + 1

Files already downloaded and verified
torch.Size([64, 3, 32, 32])
reshape output.shape:torch.Size([196608])
liner output.shape: torch.Size([10])
torch.Size([64, 3, 32, 32])
reshape output.shape:torch.Size([196608])
liner output.shape: torch.Size([10])
torch.Size([64, 3, 32, 32])
reshape output.shape:torch.Size([196608])
liner output.shape: torch.Size([10])
torch.Size([64, 3, 32, 32])
reshape output.shape:torch.Size([196608])
liner output.shape: torch.Size([10])
torch.Size([64, 3, 32, 32])
reshape output.shape:torch.Size([196608])
liner output.shape: torch.Size([10])
torch.Size([64, 3, 32, 32])
reshape output.shape:torch.Size([196608])
liner output.shape: torch.Size([10])
torch.Size([64, 3, 32, 32])
reshape output.shape:torch.Size([196608])
liner output.shape: torch.Size([10])
torch.Size([64, 3, 32, 32])
reshape output.shape:torch.Size([196608])
liner output.shape: torch.Size([10])
torch.Size([64, 3, 32, 32])
reshape output.shape:torch.Size([196608])
liner output.shape: torch.Size([10])

# 5. Sequential
![CIFAR10](/storage/pt/AW_STUDY/pytorch-tutorial-tudui/study_self/md_img/CIFAR10.png "CIFAR10")

In [18]:
# CIFAR 10 网络模型
import torch
from torch import nn
from torch.nn import Conv2d, MaxPool2d,  Linear
from torch.nn.modules.flatten import Flatten


class Tudui(nn.Module):
    # 初始化
    def __init__(self):
        # 继承父类 初始化
        super(Tudui, self).__init__()
        # 1.卷积1
        # 浅浅计算 padding和dilation应该如何设置?  stride=1,padding=2
        self.conv1 = Conv2d(in_channels=3,out_channels=32,kernel_size=5,stride=1,padding=2)
        # 2.最大池化1
        self.maxpool1 = MaxPool2d(kernel_size=2)
        # 3.卷积2
        self.conv2 = Conv2d(32,32,5,padding=2)
        # 4.最大池化2
        self.maxpool2 = MaxPool2d(kernel_size=2)
        # 5.卷积3
        self.conv3 = Conv2d(32,64,5,padding=2) # padding要写,因为padding前面一个参数是stride
        # 6 最大池化3
        self.maxpool3 = MaxPool2d(kernel_size=2)
        # 7. 通过fatten数据展平
        self.flatten = Flatten()
        # 8. 线性层1
        self.linear1 = Linear(in_features=1024,out_features=64)
        # 9. 线性层2
        self.linear2 = Linear(64,10)

    def forward(self,x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

# 实例化
tudui = Tudui()
print(tudui) # 检查网络正确性
'''
输出结果:网络结构
Tudui(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten()
  (linear1): Linear(in_features=1024, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=10, bias=True)
  )

'''

input = torch.ones((64,3,32,32))
output = tudui(input)
print(output.shape) # torch.Size([64, 10]) # 理解: 相当于有64个图片 每个图片10个东西

Tudui(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=1024, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=10, bias=True)
)
torch.Size([64, 10])


In [20]:
help(torch.nn.Sequential)

Help on class Sequential in module torch.nn.modules.container:

class Sequential(torch.nn.modules.module.Module)
 |  Sequential(*args)
 |  
 |  A sequential container.
 |  Modules will be added to it in the order they are passed in the
 |  constructor. Alternatively, an ``OrderedDict`` of modules can be
 |  passed in. The ``forward()`` method of ``Sequential`` accepts any
 |  input and forwards it to the first module it contains. It then
 |  "chains" outputs to inputs sequentially for each subsequent module,
 |  finally returning the output of the last module.
 |  
 |  The value a ``Sequential`` provides over manually calling a sequence
 |  of modules is that it allows treating the whole container as a
 |  single module, such that performing a transformation on the
 |  ``Sequential`` applies to each of the modules it stores (which are
 |  each a registered submodule of the ``Sequential``).
 |  
 |  What's the difference between a ``Sequential`` and a
 |  :class:`torch.nn.ModuleList`? A

In [ ]:
# CIFAR 10 网络模型 引入 Sequential 
import torch
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Linear, Sequential
from torch.nn.modules.flatten import Flatten
from torch.utils.tensorboard import SummaryWriter

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        # 引入 Seqential
        # self.model = Sequential(
        #     Conv2d(3,32,5,padding=2),
        #     MaxPool2d(2),
        #     Conv2d(32,32,5,padding=2),
        #     MaxPool2d(2),
        #     Conv2d(32,64,5,padding=2),
        #     MaxPool2d(2),
        #     Flatten(),
        #     Linear(1024,64),
        #     Linear(64,10)
        # )

                                            # batch, c, w, h
        self.model1 = Sequential(           # 64, 3, 32, 32
            Conv2d(3, 32, 5, padding=2),    # 64, 32, 32, 32
            MaxPool2d(2),                   # 64, 32, 16, 16
            Conv2d(32, 32, 5, padding=2),   # 64, 32, 16, 16
            MaxPool2d(2),                   # 64, 32, 8, 8
            Conv2d(32, 64, 5, padding=2),   # 64, 64, 8, 8
            MaxPool2d(2),                   # 64, 64, 4, 4
            Flatten(),                      # 64, 1024
            Linear(1024,64),                # 64, 64
            Linear(64,10)                   # 64, 10
        )

    def forward(self, x):
        x = self.model1(x)
        return x

tudui = Tudui() 
print(f"tudui: {tudui}")
input = torch.ones((64, 3, 32, 32))
output = tudui(input)
print(f"output: {output}")
print(f"output shape: {output.shape}")

# 通过tensorboard显示网络框架
writer = SummaryWriter("./net_logs")
writer.add_graph(tudui,input)
writer.close()

tudui: Tudui(
  (model1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)
output: tensor([[-0.0694, -0.0574,  0.0824,  0.0158,  0.1760, -0.0284, -0.1843,  0.0645,
          0.1832,  0.1506],
        [-0.0694, -0.0574,  0.0824,  0.0158,  0.1760, -0.0284, -0.1843,  0.0645,
          0.1832,  0.1506],
        [-0.0694, -0.0574,  0.0824,  0.0158,  0.1760, -0.0284, -0.1843,  0.0645,
         